### Data Cleaning

We scraped the data from HDB website and put it into a csv file!

Steps taken:
1. Clean dataset and convert data to be useful for feature engineering
2. Create additional features such as finding nearest distance to amenities

In [1]:
# import relevant library
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
# read csv file
url = "data.csv"
HDB_resale_df = pd.read_csv(url)

This is how our HDB resale dataset looks like right now

In [3]:
HDB_resale_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


We need to find which variables influence a HDB resale flat the most. We realized we need to clean the data and include more variables that will hopefully help us later in the mode prediction.

As our dataset consists of resale flats from 2017 to 2022, we need to account for inflation and adjust all resale price to 2022 levels. We will be using https://www.hdb.gov.sg/residential/selling-a-flat/overview/resale-statistics as of 21/12/2022

We also need to return the region based on town. This will help us in our EDA later. We will return the region based on this: https://www.hdb.gov.sg/about-us/history/hdb-towns-your-home

We also need to clean the storey range and store as a categorical variable that makes more sense for a human reader. We will return the storey category based on this: https://singaporepublichousing.fandom.com/wiki/HDB_unit_Elevation-Ground_floor,low_floor,mid-floor_and_high_floor#:~:text=Generally%2Cproperty%20agents%20define%20low,lift%20from%20the%20upper%20floors
In summary: low: 1-4, mid: 4-6 or 7 to 9, rest of the values are high

In [4]:
# function to adjust for inflation

def adjust_inflation(ind):
  t_year = HDB_resale_df['transaction_year'][ind]
  t_month = HDB_resale_df['transaction_month'][ind]
  if t_month <= 4:
    t_month = 1
  elif t_month <= 6:
    t_month = 2
  elif t_month <= 9:
    t_month = 3
  else:
    t_month = 4

  t_price = HDB_resale_df['resale_price'][ind]
  # storing inflation data
  inflation_dict = {
      "2022": {
        "4":  168.1,
        "3":	168.1,
        "2":	163.9,
        "1":	159.5
      },	
      "2021": {
        "4":	155.7,
        "3":	150.6,
        "2":	146.4,
        "1":	142.2
      },	
      "2020": {
        "4":	138.1,
        "3":	133.9,
        "2":	131.9,
        "1":	131.5
      },	 
      "2019": {
        "4":	131.5,
        "3":	130.9,
        "2":	130.8,
        "1":	131
      },	
      "2018": {
        "4":	131.4,
        "3":	131.6,
        "2":	131.7,
        "1":	131.6
      },	
      "2017": {
        "4":	132.6,
        "3":	132.8,
        "2":	133.7,
        "1":	133.9
      },	    
  }
  # adjusting price for inflation
  HDB_resale_df.loc[ind, 'adjusted_resale_price'] = t_price * ( inflation_dict['2022']['3'] / inflation_dict[ str(t_year) ][ str(t_month) ] )
  return

# function to return region based on town
def identify_region(ind):
    # return region based on town i.e. function returns NORTH if town is Bishan 
    # https://www.hdb.gov.sg/about-us/history/hdb-towns-your-home
    location = HDB_resale_df['town'][ind]
    loca = {
        'NORTH':{
                'SEMBAWANG',
                'WOODLANDS',
                'YISHUN',
            },
        'NORTH-EAST':{
                'ANG MO KIO',
                'HOUGANG',
                'PUNGGOL',
                'SENGKANG',
                'SERANGOON',
            },
        'EAST':{
                'BEDOK',
                'PASIR RIS',
                'TAMPINES',
            },
        'WEST':{
                'BUKIT BATOK',
                'BUKIT PANJANG',
                'CHOA CHU KANG',
                'CLEMENTI',
                'JURONG EAST',
                'JURONG WEST',
                'TENGAH',
            },
        'CENTRAL':{
                'BISHAN',
                'BUKIT MERAH',
                'BUKIT TIMAH',
                'CENTRAL AREA',
                'GEYLANG',
                'KALLANG/WHAMPOA',
                'MARINE PARADE',
                'QUEENSTOWN',
                'TOA PAYOH',
            }, 
        }
    for key in loca:
        if location in loca[key]:
            # add region to df
            HDB_resale_df.loc[ind, 'region'] = key
    return

# function to clean storey

def clean_storey(ind):
        # cleaning storey range
    # https://singaporepublichousing.fandom.com/wiki/HDB_unit_Elevation-Ground_floor,low_floor,mid-floor_and_high_floor#:~:text=Generally%2Cproperty%20agents%20define%20low,lift%20from%20the%20upper%20floors.
        # summary -> low: 1 - 4, mid: 5 - 8, high: >=9
        # our data is split 1 - 3, 4 - 6, 7 - 9 & more
        # new cleaning -> low: 1 - 3, mid: 4 - 9, high: >=9
    # classify storey accordingly
    level = HDB_resale_df.loc[ind, 'storey_range']
    if level == '01 TO 03':
        HDB_resale_df.loc[ind, 'storey_classification'] = 'low'
    elif level == '04 TO 06' or level == '07 TO 09':
        HDB_resale_df.loc[ind, 'storey_classification'] = 'mid'
    elif level in ['10 TO 12', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45']:
        HDB_resale_df.loc[ind, 'storey_classification'] = 'high'

In [5]:
# separating year and month from date for clarity
HDB_resale_df['adjusted_resale_price'] = 0
HDB_resale_df['transaction_year'] = pd.DatetimeIndex(HDB_resale_df['month']).year
HDB_resale_df['transaction_month'] = pd.DatetimeIndex(HDB_resale_df['month']).month
HDB_resale_df['region'] = 'null'
HDB_resale_df['storey_classification'] = 'null'
for i in range(len(HDB_resale_df)):
    # adjust for inflation
    adjust_inflation(i)
    # adjust region based on location
    identify_region(i)
    # adjust storey based on level
    clean_storey(i)
# one-hot encoding storey_classification
HDB_resale_df['storey_coded'] = HDB_resale_df['storey_classification'].replace({'high':3, 'mid':2, 'low':1})
# price order - increasing: NORTH, WEST, NORTH-EAST, EAST, CENTRAL
# one-hot encoding region based on price order
HDB_resale_df['region_coded'] = HDB_resale_df['region'].replace({'NORTH':1, 'WEST':2, 'NORTH-EAST':3, 'EAST':4, 'CENTRAL':5})

HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,adjusted_resale_price,transaction_year,transaction_month,region,storey_classification,storey_coded,region_coded
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,291256.161314,2017,1,NORTH-EAST,high,3,3
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,313853.622106,2017,1,NORTH-EAST,low,1,3
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,328918.595967,2017,1,NORTH-EAST,low,1,3
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,332684.839432,2017,1,NORTH-EAST,mid,2,3
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,332684.839432,2017,1,NORTH-EAST,low,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142418,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,842000.0,887399.373041,2022,2,NORTH,mid,2,1
142419,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,845000.0,890561.128527,2022,2,NORTH,high,3,1
142420,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,862000.0,884089.078707,2022,5,NORTH,mid,2,1
142421,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,880000.0,880000.000000,2022,9,NORTH,high,3,1


In [7]:
# saving transformed data to csv file
# HDB_resale_df.to_csv('transformed_df.csv')

We want to now get the coordinates of each individual resale flat. Coordinates data is extremely useful because it will help us in our EDA and will also help us calculate the distance between the resale flat and nearby amenities to see if there is any correlation. 

We first need to prepare the address of each resale flat

In [8]:
HDB_resale_df['address'] = HDB_resale_df['block'] + " " + HDB_resale_df['street_name']

addressList = list(HDB_resale_df['address'])
addressList

['406 ANG MO KIO AVE 10',
 '108 ANG MO KIO AVE 4',
 '602 ANG MO KIO AVE 5',
 '465 ANG MO KIO AVE 10',
 '601 ANG MO KIO AVE 5',
 '150 ANG MO KIO AVE 5',
 '447 ANG MO KIO AVE 10',
 '218 ANG MO KIO AVE 1',
 '447 ANG MO KIO AVE 10',
 '571 ANG MO KIO AVE 3',
 '534 ANG MO KIO AVE 10',
 '233 ANG MO KIO AVE 3',
 '235 ANG MO KIO AVE 3',
 '219 ANG MO KIO AVE 1',
 '536 ANG MO KIO AVE 10',
 '230 ANG MO KIO AVE 3',
 '570 ANG MO KIO AVE 3',
 '624 ANG MO KIO AVE 4',
 '441 ANG MO KIO AVE 10',
 '625 ANG MO KIO AVE 9',
 '119 ANG MO KIO AVE 3',
 '255 ANG MO KIO AVE 4',
 '432 ANG MO KIO AVE 10',
 '211 ANG MO KIO AVE 3',
 '584 ANG MO KIO AVE 3',
 '118 ANG MO KIO AVE 4',
 '333 ANG MO KIO AVE 1',
 '256 ANG MO KIO AVE 4',
 '330 ANG MO KIO AVE 1',
 '557 ANG MO KIO AVE 10',
 '302 ANG MO KIO AVE 3',
 '575 ANG MO KIO AVE 10',
 '509 ANG MO KIO AVE 8',
 '472 ANG MO KIO AVE 10',
 '475 ANG MO KIO AVE 10',
 '629 ANG MO KIO AVE 4',
 '546 ANG MO KIO AVE 10',
 '131 ANG MO KIO AVE 3',
 '254 ANG MO KIO AVE 4',
 '470 ANG MO

In [104]:
test_list = addressList[0:5]
test_list

['406 ANG MO KIO AVE 10',
 '108 ANG MO KIO AVE 4',
 '602 ANG MO KIO AVE 5',
 '465 ANG MO KIO AVE 10',
 '601 ANG MO KIO AVE 5']

Using oneMap API, we will be able to get the specific coordinates of each resale flat! https://www.onemap.gov.sg/docs/#search

In [12]:
# function for getting coordinates
import requests
import json

# def getCoordinates(address):
#   request = "https://developers.onemap.sg/commonapi/search?searchVal="+address+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"
#   response = eval(requests.get(request).text)
#   if len((response['results'])) > 0:
#     latitude = response['results'][0]['LATITUDE']
#     longitude = response['results'][0]['LONGITUDE']
#     postal = response['results'][0]['POSTAL']
#     return latitude, longitude, postal
#   else:
#     pass

def getCoordinates(address):
    url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal="+address+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"

    response = requests.request("GET", url)

    query = json.loads(response.text)
    results = query["results"]
    latitude = results[0]["LATITUDE"]
    longitude = results[0]["LONGITUDE"]
    postal = results[0]["POSTAL"]
    
    return latitude, longitude

In [107]:
import time, copy
print('Starting timer...')
start_time = time.time()

coordinatesList = []
count = 0
count1000 = 0
failed_count = 0
for address in addressList:
    try:
        if len(getCoordinates(address))>0:
            count = count + 1
            coordinatesList.append(getCoordinates(address))
            if (count == 1000):
              count = 0
              count1000 += 1
              print('Extracting',str(count1000*1000),'out of',len(addressList),'addresses')
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addressList),'addresses')
        coordinatesList.append(None)
        if failed_count == 10:
            break
print('Total Number of Addresses With No Coordinates',failed_count)
print('Stopping timer...')
time_taken = time.time() - start_time
print('Execution time ' + str(time_taken) + 'seconds.')

Starting timer...
Extracting 1000 out of 142423 addresses
Extracting 2000 out of 142423 addresses
Extracting 3000 out of 142423 addresses
Extracting 4000 out of 142423 addresses
Extracting 5000 out of 142423 addresses
Extracting 6000 out of 142423 addresses
Extracting 7000 out of 142423 addresses
Extracting 8000 out of 142423 addresses
Extracting 9000 out of 142423 addresses
Extracting 10000 out of 142423 addresses
Extracting 11000 out of 142423 addresses
Extracting 12000 out of 142423 addresses
Extracting 13000 out of 142423 addresses
Extracting 14000 out of 142423 addresses
Extracting 15000 out of 142423 addresses
Extracting 16000 out of 142423 addresses
Extracting 17000 out of 142423 addresses
Extracting 18000 out of 142423 addresses
Extracting 19000 out of 142423 addresses
Extracting 20000 out of 142423 addresses
Extracting 21000 out of 142423 addresses
Extracting 22000 out of 142423 addresses
Extracting 23000 out of 142423 addresses
Extracting 24000 out of 142423 addresses
Extract

In [126]:
latitudeList = []
longitudeList = []
postalCodeList = []
for coordinates in coordinatesList:
        latitudeList.append(coordinates[0])
        longitudeList.append(coordinates[1])
        postalCodeList.append(coordinates[2])

IndexError: tuple index out of range

In [114]:
HDB_resale_df['latitude'] = pd.DataFrame(latitudeList)
HDB_resale_df['longitude'] = pd.DataFrame(longitudeList)
HDB_resale_df['coordinates'] = list(zip(HDB_resale_df['latitude'], HDB_resale_df['longitude']))
HDB_resale_df['postal_code'] = pd.DataFrame(postalCodeList)

In [115]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,transaction_month,region,storey_classification,storey_coded,region_coded,address,latitude,longitude,coordinates,postal_code
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,1,NORTH-EAST,high,3,3,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407,"(1.36200453938712, 103.853879910407)",560406
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,1,NORTH-EAST,low,1,3,108 ANG MO KIO AVE 4,1.37096635222625,103.838201940326,"(1.37096635222625, 103.838201940326)",560108
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,1,NORTH-EAST,low,1,3,602 ANG MO KIO AVE 5,1.38070883044887,103.835368226602,"(1.38070883044887, 103.835368226602)",560602
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,1,NORTH-EAST,mid,2,3,465 ANG MO KIO AVE 10,1.3662010408294,103.857200967235,"(1.3662010408294, 103.857200967235)",560465
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,1,NORTH-EAST,low,1,3,601 ANG MO KIO AVE 5,1.38104134784496,103.835131744823,"(1.38104134784496, 103.835131744823)",560601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142418,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,2,NORTH,mid,2,1,633 YISHUN ST 61,1.41865757414383,103.839842120968,"(1.41865757414383, 103.839842120968)",760633
142419,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,2,NORTH,high,3,1,632 YISHUN ST 61,1.41856736783462,103.839186937262,"(1.41856736783462, 103.839186937262)",760632
142420,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,5,NORTH,mid,2,1,605 YISHUN ST 61,1.42172608902193,103.836336375801,"(1.42172608902193, 103.836336375801)",760605
142421,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,9,NORTH,high,3,1,633 YISHUN ST 61,1.41865757414383,103.839842120968,"(1.41865757414383, 103.839842120968)",760633


In [117]:
# saving transformed data to csv file
HDB_resale_df.to_csv('transformed_df.csv')

In [143]:
HDB_resale_df = pd.read_csv('transformed_df.csv')

In [144]:
# adding sector code

HDB_resale_df['sector_code'] = HDB_resale_df['postal_code'].str[:2]

In [145]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,region,storey_classification,storey_coded,region_coded,address,latitude,longitude,coordinates,postal_code,sector_code
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,NORTH-EAST,high,3,3,406 ANG MO KIO AVE 10,1.362005,103.853880,"('1.36200453938712', '103.853879910407')",560406,56
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,NORTH-EAST,low,1,3,108 ANG MO KIO AVE 4,1.370966,103.838202,"('1.37096635222625', '103.838201940326')",560108,56
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,NORTH-EAST,low,1,3,602 ANG MO KIO AVE 5,1.380709,103.835368,"('1.38070883044887', '103.835368226602')",560602,56
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,NORTH-EAST,mid,2,3,465 ANG MO KIO AVE 10,1.366201,103.857201,"('1.3662010408294', '103.857200967235')",560465,56
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,NORTH-EAST,low,1,3,601 ANG MO KIO AVE 5,1.381041,103.835132,"('1.38104134784496', '103.835131744823')",560601,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,NORTH,mid,2,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,NORTH,high,3,1,632 YISHUN ST 61,1.418567,103.839187,"('1.41856736783462', '103.839186937262')",760632,76
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,NORTH,mid,2,1,605 YISHUN ST 61,1.421726,103.836336,"('1.42172608902193', '103.836336375801')",760605,76
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,NORTH,high,3,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76


In [163]:
latitudeList = list(HDB_resale_df['latitude'])
longitudeList = list(HDB_resale_df['longitude'])
coordinateList = []

for i in range(len(latitudeList)):
    coordinate = (latitudeList[i],longitudeList[i])
    coordinateList.append(coordinate)

In [161]:
len(coordinateList)

142422

### Adding more distanced-based features
Let's find out other amenities of interest! Such as MRT Stations, distance to CBD, shopping malls etc.

In [85]:
stationList = [
  'Kranji',
  'Marsiling',
  'Woodlands',
  'Admiralty',
  'Sembawang',
  'Canberra',
  'Yishun',
  'Khatib',
  'Woodlands North',
  'Woodlands South',
  'Springleaf',
  'Lentor',
  'Mayflower',
  'Tuas Link',
  'Tuas West Road',
  'Tuas Crescent',
  'Gul Circle',
  'Joo Koon',
  'Pioneer',
  'Boon Lay',
  'Lakeside',
  'Chinese Garden',
  'Jurong East',
  'Clementi',
  'Dover',
  'Bukit Batok',
  'Bukit Gombak',
  'Choa Chu Kang',
  'Yew Tee',
  'Yio Chu Kang',
  'Ang Mo Kio',
  'Serangoon',
  'Bartley',
  'Kovan',
  'Hougang',
  'Buangkok',
  'Sengkang',
  'Punggol',
  'Ubi',
  'Kaki Bukit',
  'Bedok North',
  'Bedok Reservoir',
  'Tampines West',
  'Tampines',
  'Tampines East',
  'Upper Changi',
  'Expo',
  'Eunos',
  'Kembangan',
  'Bedok',
  'Tanah Merah',
  'Changi Airport',
  'Simei',
  'Pasir Ris',
  'Hillview',
  'Beauty World',
  'King Albert Park',
  'Sixth Avenue',
  'Tan Kah Kee',
  'Botanic Gardens',
  'Stevens',
  'Newton',
  'Little India',
  'Rochor',
  'Bugis',
  'Promenade',
  'Bayfront',
  'Downtown',
  'Telok Ayer',
  'Chinatown',
  'Fort Canning',
  'Bencoolen',
  'Jalan Besar',
  'Bendemeer',
  'Geylang Bahru',
  'Mattar',
  'MacPherson',
  'Buona Vista',
  'Commonwealth',
  'Queenstown',
  'Redhill',
  'Tiong Bahru',
  'Outram Park',
  'Tanjong Pagar',
  'Raffles Place',
  'City Hall',
  'Lavender',
  'Kallang',
  'Aljunied',
  'Paya Lebar',
  'HarbourFront',
  'Clarke Quay',
  'Dhoby Ghaut',
  'Farrer Park',
  'Boon Keng',
  'Potong Pasir',
  'Woodleigh',
  'Bishan',
  'Braddell',
  'Toa Payoh',
  'Novena',
  'Orchard',
  'Somerset',
  'Marina Bay',
  'Marina South Pier',
  'Bright Hill',
  'Upper Thomson',
  'Caldecott',
  'Napier',
  'Orchard Boulevard',
  'Great World',
  'Havelock',
  'Maxwell',
  'Shenton Way',
  'Gardens by the Bay',
  'Telok Blangah',
  'Labrador Park',
  'Pasir Panjang',
  'Haw Par Villa',
  'Kent Ridge',
  'one-north',
  'Holland Village',
  'Farrer Road',
  'Marymount',
  'Lorong Chuan',
  'Tai Seng',
  'Dakota',
  'Mountbatten',
  'Stadium',
  'Nicoll Highway',
  'Esplanade',
  'Bras Basah'
]

def getStationCoordinates(station):
  request = "https://developers.onemap.sg/commonapi/search?searchVal="+station+" mrt station" + "&returnGeom=Y&getAddrDetails=Y&pageNum=1"
  # print(request)
  response = eval(requests.get(request).text)
  if len((response['results'])) > 0:
    latitude = response['results'][0]['LATITUDE']
    longitude = response['results'][0]['LONGITUDE']
    return latitude, longitude
  else:
    pass
  

In [86]:
import time, copy
print('Starting timer...')
start_time = time.time()

stationCoordinatesList = []
count = 0
failed_count = 0
for station in stationList:
    try:
        if len(getStationCoordinates(station))>0:
            count = count + 1
            stationCoordinatesList.append(getStationCoordinates(station))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(stationList),'addresses')
        stationCoordinatesList.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)
print('Stopping timer...')
time_taken = time.time() - start_time
print('Execution time ' + str(time_taken) + 'seconds.')
print('Station Coordinates:')
print(stationCoordinatesList)
print('==============================')

Starting timer...
Total Number of Addresses With No Coordinates 0
Stopping timer...
Execution time 20.270045042037964seconds.
Station Coordinates:
[('1.42508698073648', '103.762137459497'), ('1.43252114855026', '103.774074641403'), ('1.43605761708128', '103.787938777173'), ('1.44058856161847', '103.800990519771'), ('1.44905082158502', '103.820046140211'), ('1.44307664075699', '103.829702590959'), ('1.42944308477331', '103.835005047246'), ('1.41738337009565', '103.832979908243'), ('1.44778217342648', '103.7851355073'), ('1.42748811362716', '103.792729941333'), ('1.39758106126824', '103.817856949947'), ('1.38550657972169', '103.835743809669'), ('1.37146318831015', '103.836567766945'), ('1.34088242451105', '103.636991425128'), ('1.32998504653102', '103.639616648771'), ('1.32102695598684', '103.649078235627'), ('1.31947090075208', '103.660530461345'), ('1.3277170408447', '103.678374996165'), ('1.33758701106708', '103.697321608474'), ('1.33860405469845', '103.706064622772'), ('1.34425911493

In [87]:
latitudeList = []
longitudeList = []
for stationCoordinates in stationCoordinatesList:
  latitudeList.append(stationCoordinates[0])
  longitudeList.append(stationCoordinates[1])

In [88]:
# Now to get LRT Stations!

LRTStationList = [
    'South View',
    'Keat Hong',
    'Teck Whye',
    'Phoenix',
    'Petir',
    'Pending',
    'Bangkit',
    'Fajar',
    'Segar',
    'Jelapang',
    'Senja',
    'Compassvale',
    'Rumbia',
    'Bakau',
    'Kangkar',
    'Ranggung',
    'Cheng Lim',
    'Farmway',
    'Kupang',
    'Thanggam',
    'Fernvale',
    'Layar',
    'Tongkang',
    'Renjong',
    'Damai',
    'Oasis',
    'Kadaloor',
    'Riviera',
    'Coral Edge',
    'Meridian',
    'Cove',
    'Sam Kee',
    'Teck Lee',
    'Punggol Point',
    'Samudera',
    'Nibong',
    'Sumang',
    'Soo Teck'
]

def getLRTStationCoordinates(station):
  request = "https://developers.onemap.sg/commonapi/search?searchVal="+station+" LRT station" + "&returnGeom=Y&getAddrDetails=Y&pageNum=1"
  # print(request)
  response = eval(requests.get(request).text)
  if len((response['results'])) > 0:
    latitude = response['results'][0]['LATITUDE']
    longitude = response['results'][0]['LONGITUDE']
    return latitude, longitude
  else:
    pass

In [89]:
import time, copy
print('Starting timer...')
start_time = time.time()

LRTstationCoordinatesList = []
count = 0
failed_count = 0
for station in LRTStationList:
    try:
        if len(getLRTStationCoordinates(station))>0:
            count = count + 1
            LRTstationCoordinatesList.append(getLRTStationCoordinates(station))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(stationList),'addresses')
        LRTstationCoordinatesList.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)
print('Stopping timer...')
time_taken = time.time() - start_time
print('Execution time ' + str(time_taken) + 'seconds.')
print('Station Coordinates:')
print(LRTstationCoordinatesList)
print('==============================')

Starting timer...
Total Number of Addresses With No Coordinates 0
Stopping timer...
Execution time 5.362050771713257seconds.
Station Coordinates:
[('1.38029828742399', '103.745291799824'), ('1.3786032506373', '103.749055668925'), ('1.37668467920808', '103.753712232337'), ('1.37861545104083', '103.757995558346'), ('1.37777204259082', '103.766645782497'), ('1.37613574334988', '103.771261175571'), ('1.38002223010088', '103.772647370452'), ('1.38457317668026', '103.770887223018'), ('1.38778508663361', '103.769599685249'), ('1.38673926997961', '103.76453408646'), ('1.38272547134453', '103.76234425472'), ('1.39449304450373', '103.900492450944'), ('1.39146849534132', '103.90597359341'), ('1.38799431054768', '103.905415300171'), ('1.38395911688913', '103.90222537044'), ('1.38423355918277', '103.897194667804'), ('1.39627763344324', '103.893797181731'), ('1.39717019591821', '103.889304824196'), ('1.39821283031379', '103.881256222255'), ('1.39731815581938', '103.875635154729'), ('1.39188589234766

In [91]:
for station in LRTStationList:
    stationList.append(station)
    
for stationCoordinates in LRTstationCoordinatesList:
  latitudeList.append(stationCoordinates[0])
  longitudeList.append(stationCoordinates[1])


In [99]:
station_df = pd.DataFrame()
station_df['station'] = pd.DataFrame(stationList)
station_df['latitude'] = pd.DataFrame(latitudeList)
station_df['longitude'] = pd.DataFrame(longitudeList)
station_df['coordinates'] = list(zip(station_df['latitude'], station_df['longitude']))
station_df.head()

# saving station data to csv file
station_df.to_csv('station_df.csv')

Now to find malls

In [28]:
#List of malls

mallList = [
    '100 AM',
    '313@Somerset',
    'Aperia',
    'Balestier Hill Shopping Centre',
    'Bugis Cube',
    'Bugis Junction',
    'Bugis+',
    'Capitol Piazza',
    'Cathay Cineleisure Orchard',
    'The Centrepoint',
    'City Square Mall',
    'CityLink Mall',
    'Duo',
    'Far East Plaza',
    'Funan',
    'Great World City',
    'HDB Hub',
    'Holland Road Shopping Centre',
    'Raffles Holland V',
    'ION Orchard',
    'Junction 8',
    'Knightsbridge',
    'Liat Towers',
    'Lucky Plaza',
    'Marina Bay Sands',
    'The Shoppes at Marina Bay Sands',
    'Marina Bay Link Mall',
    'Marina Square',
    'Millenia Walk',
    'Mustafa',
    'Ngee Ann City',
    'Orchard Central',
    'Orchard Gateway',
    'Orchard Plaza',
    'Midpoint Orchard',
    'Palais Renaissance',
    "People's Park Centre",
    "People's Park Complex",
    'Plaza Singapura',
    "GR.iD",
    'Raffles City',
    'Scotts Square',
    '350 ORCHARD ROAD SHAW HOUSE',
    'Sim Lim Square',
    'Singapore Shopping Centre',
    'The South Beach',
    'Square 2',
    'Sunshine Plaza',
    'Suntec City',
    'Tanglin Mall',
    'Tanjong Pagar Centre',
    'Tekka Centre',
    'The Adelphi',
    'The Paragon',
    'Tiong Bahru Plaza',
    'The Poiz',
    'Thomson Plaza',
    'United Square',
    'Thomson V',
    'Velocity@Novena Square',
    'Wheelock Place',
    'Wisma Atria',
    'Zhongshan Mall',
    'Bedok Mall',
    'Century Square',
    'Our Tampines Hub',
    'Changi City Point',
    'Downtown East',
    'Djitsun Mall Bedok',
    'Eastpoint Mall',
    'Jewel Changi Airport',
    'KINEX',
    'Katong Shopping Centre',
    'Katong Square',
    'Kallang Wave Mall',
    'Leisure Park Kallang',
    'i12 Katong',
    'Parkway Parade',
    'Paya Lebar Square',
    'Paya Lebar Quarter',
    'Roxy Square',
    'Singpost Centre',
    'Tampines 1',
    'Tampines Mall',
    'White Sands',
    'City Plaza',
    'Elias Mall',
    'Loyang Point',
    '888 Plaza',
    'Admiralty Place',
    'AMK Hub',
    'Canberra Plaza',
    'Causeway Point',
    'Woodlands Civic Centre',
    'Broadway Plaza',
    'Djitsun Mall',
    'Jubilee Square',
    'Junction 8',
    'Junction Nine',
    'Marsiling Mall',
    'Northpoint City',
    'Sembawang Shopping Centre',
    'Sun Plaza',
    'Vista Point',
    'Wisteria Mall',
    'Woodlands Mart',
    'Woodlands North Plaza',
    'Waterway Point',
    'Compass One',
    'Hougang Mall',
    'Heartland Mall',
    'NEX',
    'Buangkok Square',
    'Greenwich V',
    'Hougang 1',
    'Hougang Green Shopping Mall',
    'Hougang Rivercourt',
    'myVillage At Serangoon Garden',
    'Northshore Plaza',
    'Oasis Terraces',
    'Punggol Plaza',
    'Rivervale Mall',
    'Rivervale Plaza',
    'The Seletar Mall',
    'Upper Serangoon Shopping Centre',
    'Beauty World Centre',
    'Beauty World Plaza',
    'Bukit Panjang Plaza',
    'Bukit Timah Plaza',
    'Fajar Shopping Centre',
    'Greenridge Shopping Centre',
    'Hillion Mall',
    'HillV2',
    'Junction 10',
    'Keat Hong Shopping Centre',
    'Limbang Shopping Centre',
    'Lot One',
    'Rail Mall',
    'Sunshine Place',
    'Teck Whye Shopping Centre',
    'West Mall',
    'Yew Tee Point',
    'Yew Tee Square',
    'VivoCity',
    'HarbourFront Centre',
    'Alexandra Retail Centre',
    '321 Clementi',
    'The Clementi Mall',
    'IMM',
    'Jem',
    'Westgate',
    'Jurong Point',
    'Pioneer Mall',
    'The Star Vista',
    'Alexandra Central',
    'Anchorpoint',
    'The Grandstand',
    'Boon Lay Shopping Centre',
    'Grantral Mall',
    'Fairprice Hub',
    'Gek Poh Shopping Centre',
    'Rochester Mall',
    'Taman Jurong Shopping Centre',
    'West Coast Plaza',
    'Queensway Shopping Centre'
]

In [29]:
mallCoordinatesList = []
count = 0
failed_count = 0
for mall in mallList:
    try:
        mall_coordinates = getCoordinates(mall)
        count += 1
        mallCoordinatesList.append(mall_coordinates)
    except:
        count += 1
        failed_count += 1
        print(mall)
        print('Failed to extract',count,'out of',len(mallList),'addresses')
        mallCoordinatesList.append(np.nan)
        
print('Total Number of Addresses With No Coordinates',failed_count)
print('Mall Coordinates:')
print(mallCoordinatesList)
print('==============================')

Total Number of Addresses With No Coordinates 0
Mall Coordinates:
[('1.27458821795426', '103.84347073661'), ('1.30100656917241', '103.838246592796'), ('1.31086672816827', '103.863929573292'), ('1.32559594839311', '103.842571612968'), ('1.2981408343975', '103.855635339249'), ('1.29911287475873', '103.855411173758'), ('1.30095171530648', '103.855172625542'), ('1.29307884763132', '103.851261982149'), ('1.30149264852924', '103.836406753067'), ('1.30197836876806', '103.839759023144'), ('1.31142103107683', '103.856624019991'), ('1.2927777312893', '103.854173501417'), ('1.29953434891664', '103.85840168774'), ('1.30738819608479', '103.833803262332'), ('1.29134759697794', '103.849989790085'), ('1.2934759620704', '103.831975865363'), ('1.33201387005935', '103.84984949662'), ('1.31027747574118', '103.795371163103'), ('1.31056351475336', '103.796107173318'), ('1.30419142102398', '103.831510144742'), ('1.35031600039774', '103.848496645294'), ('1.3029831225958', '103.836283923443'), ('1.305087227108

In [27]:
latitudeList = []
longitudeList = []
for mallCoordinates in mallCoordinatesList:
    latitudeList.append(mallCoordinates[0])
    longitudeList.append(mallCoordinates[1])
    
print(latitudeList)

('1.27458821795426', '103.84347073661')
1.27458821795426
('1.30100656917241', '103.838246592796')
1.30100656917241
('1.31086672816827', '103.863929573292')
1.31086672816827
('1.32559594839311', '103.842571612968')
1.32559594839311
('1.2981408343975', '103.855635339249')
1.2981408343975
('1.29911287475873', '103.855411173758')
1.29911287475873
('1.30095171530648', '103.855172625542')
1.30095171530648
('1.29307884763132', '103.851261982149')
1.29307884763132
('1.30149264852924', '103.836406753067')
1.30149264852924
('1.30197836876806', '103.839759023144')
1.30197836876806
('1.31142103107683', '103.856624019991')
1.31142103107683
('1.2927777312893', '103.854173501417')
1.2927777312893
('1.29953434891664', '103.85840168774')
1.29953434891664
('1.30730468891062', '103.833818438639')
1.30730468891062
('1.29134759697794', '103.849989790085')
1.29134759697794
('1.2934759620704', '103.831975865363')
1.2934759620704
('1.33201387005935', '103.84984949662')
1.33201387005935
('1.31027747574118', '1

TypeError: 'float' object is not subscriptable

In [32]:
latitudeList = []
longitudeList = []
for mallCoordinates in mallCoordinatesList:
  latitudeList.append(float(mallCoordinates[0]))
  longitudeList.append(float(mallCoordinates[1]))
    
Mall_df = pd.DataFrame()
Mall_df['mall'] = pd.DataFrame(mallList)
Mall_df['latitude'] = pd.DataFrame(latitudeList)
Mall_df['longitude'] = pd.DataFrame(longitudeList)
Mall_df['coordinates'] = list(zip(Mall_df['latitude'], Mall_df['longitude']))

# saving MRT data to csv file
Mall_df.to_csv('Mall_df.csv')

Calculating distance from resale flat to areas of interest (eg CBD, MRT Stations, Shopping Malls)

With latitude and longitude of two given areas, we are able to calculate their distance. However, we cannot use euclidean distance as the earth is a sphere. Hence, we need to use haversine formula instead to determine the great-circle distance between two points.

To calculate distance from CBD, we will assume Raffles MRT Station as CBD

In [47]:
!pip install haversine

import haversine as hs

In [147]:
coordinatesList

["('1.36200453938712', '103.853879910407')",
 "('1.37096635222625', '103.838201940326')",
 "('1.38070883044887', '103.835368226602')",
 "('1.3662010408294', '103.857200967235')",
 "('1.38104134784496', '103.835131744823')",
 "('1.37680678506745', '103.842017962576')",
 "('1.36750371103856', '103.855897952833')",
 "('1.36511907555698', '103.841742483384')",
 "('1.36750371103856', '103.855897952833')",
 "('1.37005500420092', '103.854880566919')",
 "('1.37405846295585', '103.854168170426')",
 "('1.36754924977907', '103.836740628478')",
 "('1.36682360872342', '103.83649123351')",
 "('1.36598198004505', '103.840654039612')",
 "('1.37447378519738', '103.854100515826')",
 "('1.36885445228086', '103.837896397823')",
 "('1.3697435855576', '103.856023138277')",
 "('1.38156171229824', '103.84000226873')",
 "('1.36605027829032', '103.854168309853')",
 "('1.38219867741921', '103.841211533475')",
 "('1.36956344301912', '103.844698910817')",
 "('1.37065487078854', '103.834630215787')",
 "('1.36830996

In [166]:
# initialize all coordinates of resale flats into a list
distanceList = []

for i in range(len(latitudeList)):
    distanceList.append(hs.haversine((latitudeList[i],longitudeList[i]),(1.28393326234538, 103.851463066212)))

HDB_resale_df['distance_from_CBD'] = pd.DataFrame(distanceList)

In [167]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,storey_classification,storey_coded,region_coded,address,latitude,longitude,coordinates,postal_code,sector_code,distance_from_CBD
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,high,3,3,406 ANG MO KIO AVE 10,1.362005,103.853880,"('1.36200453938712', '103.853879910407')",560406,56,8.685298
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,low,1,3,108 ANG MO KIO AVE 4,1.370966,103.838202,"('1.37096635222625', '103.838201940326')",560108,56,9.789287
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,low,1,3,602 ANG MO KIO AVE 5,1.380709,103.835368,"('1.38070883044887', '103.835368226602')",560602,56,10.908694
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,mid,2,3,465 ANG MO KIO AVE 10,1.366201,103.857201,"('1.3662010408294', '103.857200967235')",560465,56,9.169984
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,low,1,3,601 ANG MO KIO AVE 5,1.381041,103.835132,"('1.38104134784496', '103.835131744823')",560601,56,10.949497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,mid,2,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,high,3,1,632 YISHUN ST 61,1.418567,103.839187,"('1.41856736783462', '103.839186937262')",760632,76,15.032721
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,mid,2,1,605 YISHUN ST 61,1.421726,103.836336,"('1.42172608902193', '103.836336375801')",760605,76,15.413881
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,high,3,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277


In [171]:
station_df

stationLatitudeList = list(station_df['latitude'])
stationLongitudeList = list(station_df['longitude'])
stationList = list(station_df['station'])

In [222]:
Mall_df

mallLatitudeList = list(Mall_df['latitude'])
mallLongitudeList = list(Mall_df['longitude'])
mallList = list(Mall_df['mall'])

In [223]:
# function for returning resale flat's nearest MRT and distance

def nearestMRT(coordinate):
    min = 99999999
    station = ""
    for i, b in enumerate(stationLatitudeList):
        distance = hs.haversine(coordinate, (float(stationLatitudeList[i]), float(stationLongitudeList[i])))
        if (min > distance):
            min = distance
            station = stationList[i]
    return min, station

# function for returning resale flat's nearest mall and distance

def nearestMall(coordinate):
    min = 99999999
    mall = ""
    for i, b in enumerate(mallList):
        distance = hs.haversine(coordinate, (float(mallLatitudeList[i]), float(mallLongitudeList[i])))
        if (min > distance):
            min = distance
            mall = mallList[i]
    return min, mall

In [212]:
import time, copy
print('Starting timer...')
start_time = time.time()
count = 0
count = 100

minList = []
nearestStationList = []

for i in range(len(latitudeList)):
    coordinate = (latitudeList[i], longitudeList[i])
    minList.append(nearestMRT(coordinate)[0])
    nearestStationList.append(nearestMRT(coordinate)[1])
    
print('Stopping timer...')
time_taken = time.time() - start_time
print('Execution time ' + str(time_taken) + 'seconds.')

Starting timer...
Stopping timer...
Execution time 155.3398642539978seconds.


In [215]:
HDB_resale_df['nearest_station'] = pd.DataFrame(nearestStationList)
HDB_resale_df['nearest_station_distance'] = pd.DataFrame(minList)

In [216]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,region_coded,address,latitude,longitude,coordinates,postal_code,sector_code,distance_from_CBD,nearest_station,nearest_station_distance
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,3,406 ANG MO KIO AVE 10,1.362005,103.853880,"('1.36200453938712', '103.853879910407')",560406,56,8.685298,Ang Mo Kio,0.960938
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,3,108 ANG MO KIO AVE 4,1.370966,103.838202,"('1.37096635222625', '103.838201940326')",560108,56,9.789287,Mayflower,0.189871
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,3,602 ANG MO KIO AVE 5,1.380709,103.835368,"('1.38070883044887', '103.835368226602')",560602,56,10.908694,Lentor,0.535118
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,3,465 ANG MO KIO AVE 10,1.366201,103.857201,"('1.3662010408294', '103.857200967235')",560465,56,9.169984,Ang Mo Kio,0.932844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,3,601 ANG MO KIO AVE 5,1.381041,103.835132,"('1.38104134784496', '103.835131744823')",560601,56,10.949497,Lentor,0.501153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,1,632 YISHUN ST 61,1.418567,103.839187,"('1.41856736783462', '103.839186937262')",760632,76,15.032721,Khatib,0.702424
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,1,605 YISHUN ST 61,1.421726,103.836336,"('1.42172608902193', '103.836336375801')",760605,76,15.413881,Khatib,0.610240
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,1,633 YISHUN ST 61,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855


In [225]:
import time, copy
print('Starting timer...')
start_time = time.time()
count = 0
count = 100

nearestMallDistanceList = []
nearestMallList = []

for i in range(len(latitudeList)):
    coordinate = (latitudeList[i], longitudeList[i])
    nearestMallDistanceList.append(nearestMall(coordinate)[0])
    nearestMallList.append(nearestMall(coordinate)[1])
    
print('Stopping timer...')
time_taken = time.time() - start_time
print('Execution time ' + str(time_taken) + 'seconds.')

Starting timer...
Stopping timer...
Execution time 126.2641396522522seconds.


In [226]:
HDB_resale_df['nearest_mall'] = pd.DataFrame(nearestMallList)
HDB_resale_df['nearest_mall_distance'] = pd.DataFrame(nearestMallDistanceList)

HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,latitude,longitude,coordinates,postal_code,sector_code,distance_from_CBD,nearest_station,nearest_station_distance,nearest_mall,nearest_mall_distance
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,1.362005,103.853880,"('1.36200453938712', '103.853879910407')",560406,56,8.685298,Ang Mo Kio,0.960938,AMK Hub,1.017286
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,1.370966,103.838202,"('1.37096635222625', '103.838201940326')",560108,56,9.789287,Mayflower,0.189871,Broadway Plaza,0.867983
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,1.380709,103.835368,"('1.38070883044887', '103.835368226602')",560602,56,10.908694,Lentor,0.535118,Broadway Plaza,1.528024
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,1.366201,103.857201,"('1.3662010408294', '103.857200967235')",560465,56,9.169984,Ang Mo Kio,0.932844,myVillage At Serangoon Garden,0.892900
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,1.381041,103.835132,"('1.38104134784496', '103.835131744823')",560601,56,10.949497,Lentor,0.501153,Broadway Plaza,1.571906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,1.418567,103.839187,"('1.41856736783462', '103.839186937262')",760632,76,15.032721,Khatib,0.702424,Wisteria Mall,0.245374
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,1.421726,103.836336,"('1.42172608902193', '103.836336375801')",760605,76,15.413881,Khatib,0.610240,Northpoint City,0.687057
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,1.418658,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017


In [232]:
HDB_resale_df['adjusted_price_per_sqm'] = HDB_resale_df['adjusted_resale_price'] / HDB_resale_df['floor_area_sqm']

In [233]:
HDB_resale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142422 entries, 0 to 142421
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   month                     142422 non-null  object 
 1   town                      142422 non-null  object 
 2   flat_type                 142422 non-null  object 
 3   block                     142422 non-null  object 
 4   street_name               142422 non-null  object 
 5   storey_range              142422 non-null  object 
 6   floor_area_sqm            142422 non-null  float64
 7   flat_model                142422 non-null  object 
 8   lease_commence_date       142422 non-null  int64  
 9   remaining_lease           142422 non-null  object 
 10  resale_price              142422 non-null  float64
 11  adjusted_resale_price     142422 non-null  float64
 12  transaction_year          142422 non-null  int64  
 13  transaction_month         142422 non-null  i

In [234]:
HDB_resale_df.to_csv('transformed_df_features.csv')

In [3]:
# read csv file
url = 'transformed_df_features.csv'
HDB_resale_df = pd.read_csv(url)

In [4]:
HDB_resale_df.drop(axis=1, columns="Unnamed: 0", inplace=True)
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,longitude,coordinates,postal_code,sector_code,distance_from_CBD,nearest_station,nearest_station_distance,nearest_mall,nearest_mall_distance,adjusted_price_per_sqm
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,103.853880,"('1.36200453938712', '103.853879910407')",560406,56,8.685298,Ang Mo Kio,0.960938,AMK Hub,1.017286,6619.458211
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,103.838202,"('1.37096635222625', '103.838201940326')",560108,56,9.789287,Mayflower,0.189871,Broadway Plaza,0.867983,4684.382419
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,103.835368,"('1.38070883044887', '103.835368226602')",560602,56,10.908694,Lentor,0.535118,Broadway Plaza,1.528024,4909.232776
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,103.857201,"('1.3662010408294', '103.857200967235')",560465,56,9.169984,Ang Mo Kio,0.932844,myVillage At Serangoon Garden,0.892900,4892.424109
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,103.835132,"('1.38104134784496', '103.835131744823')",560601,56,10.949497,Lentor,0.501153,Broadway Plaza,1.571906,4965.445364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5189.470018
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,103.839187,"('1.41856736783462', '103.839186937262')",760632,76,15.032721,Khatib,0.702424,Wisteria Mall,0.245374,5430.250784
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,103.836336,"('1.42172608902193', '103.836336375801')",760605,76,15.413881,Khatib,0.610240,Northpoint City,0.687057,5423.859379
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,103.839842,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5365.853659


In [5]:
HDB_resale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142422 entries, 0 to 142421
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   month                     142422 non-null  object 
 1   town                      142422 non-null  object 
 2   flat_type                 142422 non-null  object 
 3   block                     142422 non-null  object 
 4   street_name               142422 non-null  object 
 5   storey_range              142422 non-null  object 
 6   floor_area_sqm            142422 non-null  float64
 7   flat_model                142422 non-null  object 
 8   lease_commence_date       142422 non-null  int64  
 9   remaining_lease           142422 non-null  object 
 10  resale_price              142422 non-null  float64
 11  adjusted_resale_price     142422 non-null  float64
 12  transaction_year          142422 non-null  int64  
 13  transaction_month         142422 non-null  i

#### Perform minor typecasting

In [6]:
# Convert int variables to category
HDB_resale_df['postal_code'] = HDB_resale_df['postal_code'].astype('category')
HDB_resale_df['sector_code'] = HDB_resale_df['sector_code'].astype('category')
HDB_resale_df['storey_coded'] = HDB_resale_df['storey_coded'].astype('category')
HDB_resale_df['region_coded'] = HDB_resale_df['region_coded'].astype('category')

HDB_resale_df.dtypes[HDB_resale_df.dtypes == 'int64']

lease_commence_date    int64
transaction_year       int64
transaction_month      int64
dtype: object

In [7]:
HDB_resale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142422 entries, 0 to 142421
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   month                     142422 non-null  object  
 1   town                      142422 non-null  object  
 2   flat_type                 142422 non-null  object  
 3   block                     142422 non-null  object  
 4   street_name               142422 non-null  object  
 5   storey_range              142422 non-null  object  
 6   floor_area_sqm            142422 non-null  float64 
 7   flat_model                142422 non-null  object  
 8   lease_commence_date       142422 non-null  int64   
 9   remaining_lease           142422 non-null  object  
 10  resale_price              142422 non-null  float64 
 11  adjusted_resale_price     142422 non-null  float64 
 12  transaction_year          142422 non-null  int64   
 13  transaction_month         142

In [8]:
HDB_resale_df['storey_range'].unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45'], dtype=object)

In [9]:
storey_number_list = []

for storey in list(HDB_resale_df['storey_range']):
    if storey == '01 TO 03':
        storey_number_list.append(2)
    elif storey == '04 TO 06':
        storey_number_list.append(5)
    elif storey == '07 TO 09':
        storey_number_list.append(8)
    elif storey == '10 TO 12':
        storey_number_list.append(11)
    elif storey == '13 TO 15':
        storey_number_list.append(14)
    elif storey == '16 TO 18':
        storey_number_list.append(17)
    elif storey == '19 TO 21':
        storey_number_list.append(20)
    elif storey == '22 TO 24':
        storey_number_list.append(23)
    elif storey == '25 TO 27':
        storey_number_list.append(26)
    elif storey == '28 TO 30':
        storey_number_list.append(29)
    elif storey == '31 TO 33':
        storey_number_list.append(32)
    elif storey == '34 TO 36':
        storey_number_list.append(35)
    elif storey == '37 TO 39':
        storey_number_list.append(38)
    elif storey == '40 TO 42':
        storey_number_list.append(41)
    elif storey == '43 TO 45':
        storey_number_list.append(44)
    elif storey == '46 TO 48':
        storey_number_list.append(47)
    else:
        storey_number_list.append(50)

storey_number_list

[11,
 2,
 2,
 5,
 2,
 2,
 5,
 5,
 5,
 2,
 2,
 11,
 5,
 8,
 8,
 5,
 11,
 5,
 8,
 5,
 8,
 5,
 11,
 2,
 5,
 8,
 8,
 11,
 8,
 11,
 11,
 8,
 2,
 11,
 8,
 2,
 2,
 2,
 5,
 5,
 5,
 5,
 5,
 11,
 11,
 11,
 14,
 20,
 23,
 2,
 14,
 20,
 14,
 11,
 11,
 8,
 5,
 5,
 8,
 2,
 2,
 5,
 5,
 5,
 11,
 14,
 11,
 5,
 11,
 11,
 14,
 8,
 14,
 8,
 5,
 8,
 2,
 8,
 5,
 8,
 11,
 5,
 8,
 11,
 5,
 11,
 2,
 5,
 5,
 14,
 5,
 11,
 5,
 14,
 2,
 5,
 8,
 14,
 2,
 8,
 8,
 8,
 2,
 5,
 8,
 14,
 11,
 5,
 5,
 14,
 8,
 5,
 11,
 2,
 8,
 5,
 17,
 17,
 14,
 5,
 2,
 14,
 5,
 2,
 11,
 11,
 11,
 11,
 5,
 5,
 5,
 5,
 5,
 2,
 8,
 8,
 2,
 14,
 35,
 5,
 2,
 8,
 23,
 8,
 8,
 5,
 5,
 5,
 2,
 5,
 5,
 11,
 2,
 5,
 2,
 8,
 8,
 5,
 8,
 8,
 2,
 8,
 5,
 2,
 5,
 8,
 11,
 11,
 11,
 5,
 8,
 11,
 2,
 17,
 5,
 8,
 2,
 8,
 2,
 5,
 2,
 2,
 14,
 17,
 14,
 8,
 11,
 23,
 11,
 14,
 8,
 14,
 11,
 2,
 14,
 8,
 8,
 8,
 8,
 5,
 5,
 11,
 11,
 2,
 2,
 5,
 5,
 11,
 8,
 5,
 8,
 8,
 8,
 5,
 8,
 2,
 5,
 14,
 2,
 2,
 2,
 23,
 2,
 5,
 8,
 5,
 5,
 11,
 14,
 17,
 5,
 2,


In [10]:
HDB_resale_df['storey_avg'] = pd.DataFrame(storey_number_list)

In [11]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,coordinates,postal_code,sector_code,distance_from_CBD,nearest_station,nearest_station_distance,nearest_mall,nearest_mall_distance,adjusted_price_per_sqm,storey_avg
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,"('1.36200453938712', '103.853879910407')",560406,56,8.685298,Ang Mo Kio,0.960938,AMK Hub,1.017286,6619.458211,11
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,"('1.37096635222625', '103.838201940326')",560108,56,9.789287,Mayflower,0.189871,Broadway Plaza,0.867983,4684.382419,2
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,"('1.38070883044887', '103.835368226602')",560602,56,10.908694,Lentor,0.535118,Broadway Plaza,1.528024,4909.232776,2
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,"('1.3662010408294', '103.857200967235')",560465,56,9.169984,Ang Mo Kio,0.932844,myVillage At Serangoon Garden,0.892900,4892.424109,5
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,"('1.38104134784496', '103.835131744823')",560601,56,10.949497,Lentor,0.501153,Broadway Plaza,1.571906,4965.445364,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5189.470018,5
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,"('1.41856736783462', '103.839186937262')",760632,76,15.032721,Khatib,0.702424,Wisteria Mall,0.245374,5430.250784,11
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,"('1.42172608902193', '103.836336375801')",760605,76,15.413881,Khatib,0.610240,Northpoint City,0.687057,5423.859379,5
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,"('1.41865757414383', '103.839842120968')",760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5365.853659,11


In [12]:
years = HDB_resale_df['remaining_lease'].str.slice(start=0, stop=2)
months = HDB_resale_df['remaining_lease'].str.slice(start=9, stop=11)

In [13]:
remaining_months_list = []

for lease in HDB_resale_df['remaining_lease']:
    years = int(lease[0:2])
    months = lease[9:11]

    if (months == '01'):
        remaining_months_list.append(years*12+1)
    elif months == '02':
        remaining_months_list.append(years*12+2)
    elif months == '03':
        remaining_months_list.append(years*12+3)
    elif months == '04':
        remaining_months_list.append(years*12+4)
    elif months == '05':
        remaining_months_list.append(years*12+5)
    elif months == '06':
        remaining_months_list.append(years*12+6)
    elif months == '07':
        remaining_months_list.append(years*12+7)
    elif months == '08':
        remaining_months_list.append(years*12+8)
    elif months == '09':
        remaining_months_list.append(years*12+9)
    elif months == '10':
        remaining_months_list.append(years*12+10)
    elif months == '11':
        remaining_months_list.append(years*12+11)
    else:
        remaining_months_list.append(years*12)

In [14]:
HDB_resale_df['remaining_months'] = pd.DataFrame(remaining_months_list)

In [15]:
HDB_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,postal_code,sector_code,distance_from_CBD,nearest_station,nearest_station_distance,nearest_mall,nearest_mall_distance,adjusted_price_per_sqm,storey_avg,remaining_months
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,560406,56,8.685298,Ang Mo Kio,0.960938,AMK Hub,1.017286,6619.458211,11,736
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,560108,56,9.789287,Mayflower,0.189871,Broadway Plaza,0.867983,4684.382419,2,727
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,560602,56,10.908694,Lentor,0.535118,Broadway Plaza,1.528024,4909.232776,2,749
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,560465,56,9.169984,Ang Mo Kio,0.932844,myVillage At Serangoon Garden,0.892900,4892.424109,5,745
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,560601,56,10.949497,Lentor,0.501153,Broadway Plaza,1.571906,4965.445364,2,749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142417,2022-02,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,171.0,Multi Generation,1987,64 years 11 months,...,760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5189.470018,5,779
142418,2022-02,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 09 months,...,760632,76,15.032721,Khatib,0.702424,Wisteria Mall,0.245374,5430.250784,11,777
142419,2022-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,04 TO 06,163.0,Multi Generation,1988,64 years 09 months,...,760605,76,15.413881,Khatib,0.610240,Northpoint City,0.687057,5423.859379,5,777
142420,2022-09,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,64 years 04 months,...,760633,76,15.036277,Khatib,0.775855,Wisteria Mall,0.180017,5365.853659,11,772


In [16]:
HDB_resale_df.to_csv('transformed_df_features2.csv', index = False)